In [10]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import os
import numpy as np

import pickle
from tqdm import tqdm
import rdflib
from rdflib import Graph
from rdflib.namespace import BRICK

### Practice of SPARQL

In [11]:
train_mapper_path = "./datasets/bts_site_b_train/mapper_TrainOnly.csv"
train_file_path =  "./datasets/train/" 

In [12]:
df_mapper = pd.read_csv(train_mapper_path)
df_mapper.head()

,Unnamed: 0,Building,StreamID,Filename,strBrickLabel
0,0,A,9ba955fa_5960_4c9b_b73a_10156da7d083,trainAll_0.pkl,Operating_Mode_Status
1,1,A,7a027f80_7d78_4a88_a1c1_20aaed374d97,FILE NOT SAVED: y is Point.,Point
2,2,A,8fd6e75b_88bc_4992_b420_77389969b3c4,trainAll_1.pkl,Mode_Command
3,3,A,8db6eaa9_bd6c_4f7e_aed0_a47e4e192a6c,trainAll_2.pkl,Active_Power_Sensor
4,4,A,b2338dec_110a_45cc_8358_1171aaef2c45,trainAll_3.pkl,System_Status


In [13]:
filename_ttl='datasets/bts_site_b_train/Site_B.ttl'

g = Graph().parse(filename_ttl)

# Define the SPARQL query to retrieve electrical energy sensors ordered by meters
query_energy = """
SELECT ?meter ?sensor ?stream_id ?phase_count ?phases ?unit ?power_complexity ?power_flow
WHERE {
    ?sensor rdf:type brick:Electrical_Energy_Sensor .
    ?meter rdf:type brick:Electrical_Meter .
    ?sensor brick:isPointOf ?meter .
    ?sensor senaps:stream_id ?stream_id .
    OPTIONAL { ?sensor brick:electricalPhaseCount [ brick:value ?phase_count ] . }
    OPTIONAL { ?sensor brick:electricalPhases [ brick:value ?phases ] . }
    OPTIONAL { ?sensor brick:hasUnit [ brick:value ?unit ] . }
    OPTIONAL { ?sensor brick:powerComplexity [ brick:value ?power_complexity ] . }
    OPTIONAL { ?sensor brick:powerFlow [ brick:value ?power_flow ] . }
}
ORDER BY ?meter
"""

# Run the query
results_energy = g.query(query_energy)

# Prepare the data for the DataFrame
data_energy = []
for row in results_energy:
    data_energy.append({
        'Meter': str(row['meter']),
        'Sensor': str(row['sensor']),
        'Stream ID': str(row['stream_id']),
        'Phase Count': str(row['phase_count']) if row['phase_count'] else None,
        'Phases': str(row['phases']) if row['phases'] else None,
        'Unit': str(row['unit']) if row['unit'] else None,
        'Power Complexity': str(row['power_complexity']) if row['power_complexity'] else None,
        'Power Flow': str(row['power_flow']) if row['power_flow'] else None
    })

# Create a DataFrame from the results
df_energy_meters = pd.DataFrame(data_energy)
df_energy_meters.to_csv('kripa_energy_meters.csv', index=False)

# Display the DataFrame
# df_energy_meters.head()
display(df_energy_meters.head())

,Meter,Sensor,Stream ID,Phase Count,Phases,Unit,Power Complexity,Power Flow
0,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,52333345_fda6_498a_a5eb_8c9becf3996b,3,ABC,KiloV-A_Reactive,reactive,net
1,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,9ff52a4f_128b_4c44_86b3_4fa1d029506e,3,ABC,KiloW-HR,real,net
2,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,40697530_1df5_47dd_ae18_89c4aaa2a8fd,3,ABC,KiloW-HR,real,net
3,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,1afde7d7_004d_41c5_89b2_ede5967e4807,3,ABC,KiloV-A_Reactive,reactive,net
4,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,d8449728_9f2d_469b_973b_2a5fa2b7d0c0,3,ABC,KiloW-HR,real,net


dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#1af5f6f9_15b1_4a62_962e_0ede16373202.983cbacc_7d0f_47e8_aa19_a0912eab6f3c

It's called URI and it can be interpreted as:

    Organization (dch
    ) → Site → Building → Sensor instance (fragment after #)
    The part after # uniquely identifies a specific instance or value, perhaps related to a measurement within that sensor.

### Outside air temp sensor (without weather station as a location)

In [14]:
filename_ttl='datasets/bts_site_b_train/Site_B.ttl'

g = Graph().parse(filename_ttl)

# Define the SPARQL query to retrieve Outside_Air_Temperature_Sensor (without using weather station as a location)
query_energy = """
SELECT ?sensor ?stream_id 
WHERE {
    ?sensor rdf:type brick:Outside_Air_Temperature_Sensor .
    ?sensor senaps:stream_id ?stream_id .
}
ORDER BY ?stream_id
"""

# Run the query
results_energy = g.query(query_energy)

# Prepare the data for the DataFrame
data_energy = []
for row in results_energy:
    data_energy.append({
        'Sensor': str(row['sensor']),
        'Stream ID': str(row['stream_id']),
    })

# Create a DataFrame from the results
df_energy_meters = pd.DataFrame(data_energy)

# Display the DataFrame
# df_energy_meters.head()
display(df_energy_meters.head())

,Sensor,Stream ID
0,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,1f24bcff_00c1_4843_b4ee_e33a4cf23c32
1,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,59d9b366_e56a_4faf_88c3_dd4299b51ba8
2,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,664ddc27_ccc5_43a1_97b7_3000c0e948db
3,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,6b5a7bdf_b566_44cf_99e8_be7ca514fa00
4,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,9c4a31dc_2195_4f59_b3eb_2d744f1bddc0


### Outside air temp sensor (weather station)

In [15]:
filename_ttl='datasets/bts_site_b_train/Site_B.ttl'
g = Graph().parse(filename_ttl)
# Define the SPARQL query to retrieve Outside_Air_Temperature_Sensor
query_energy = """
SELECT ?sensor ?stream_id 
WHERE {
    ?sensor rdf:type brick:Outside_Air_Temperature_Sensor .
    ?sensor brick:isPointOf   ?loc .
    ?loc a brick:Weather_Station .
    ?sensor senaps:stream_id ?stream_id .
}
ORDER BY ?stream_id
"""
# Run the query
results_energy = g.query(query_energy)
# Prepare the data for the DataFrame
data_energy = []
for row in results_energy:
    data_energy.append({
        'Sensor': str(row['sensor']),
        'Stream ID': str(row['stream_id']),
    })
# Create a DataFrame from the results
df_energy_meters = pd.DataFrame(data_energy)
# Display the DataFrame
display(df_energy_meters.head())

,Sensor,Stream ID
0,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,664ddc27_ccc5_43a1_97b7_3000c0e948db


In [16]:
# Define the SPARQL query to retrieve Air_Temperature_Sensor and Room_Air_Temperature_Setpoint
query_ats = '''
    SELECT ?ats_sid ?atsp_sid WHERE {
        ?ats  a                 brick:Air_Temperature_Sensor .
        ?ats  brick:isPointOf   ?room_uri .
        ?ats  senaps:stream_id  ?ats_sid .
        ?atsp a                 brick:Room_Air_Temperature_Setpoint .
        ?atsp brick:isPointOf   ?room_uri .
        ?atsp senaps:stream_id  ?atsp_sid .
    }
'''

# Run the query
results_ats = g.query(query_ats)

# Prepare the data for the DataFrame
data_ats = []
for row in results_ats:
    data_ats.append({
        'Air_Temperature_Sensor_Stream_ID': str(row['ats_sid']),
        'Setpoint_Stream_ID': str(row['atsp_sid']),
    })

# Create a DataFrame from the results
df_ats = pd.DataFrame(data_ats)

# Display the DataFrame
display(df_ats.head())

# Save the DataFrame to a CSV file
df_ats.to_csv('air_temperature_sensors.csv', index=False)
print("Data saved to 'air_temperature_sensors.csv'")


,Air_Temperature_Sensor_Stream_ID,Setpoint_Stream_ID
0,4c8df37a_3677_4c84_b346_0b6979ccbf06,58e9c3f3_0100_446a_b446_521ba738db46
1,9f6e47c8_ec19_402e_b27b_4551655ec984,58e9c3f3_0100_446a_b446_521ba738db46
2,3579ef08_6cda_483a_8b7e_cb2e471de630,67da0763_fcd4_4e68_85dd_3e3deac4f2ac
3,3ebacdfd_bd8a_44da_9241_f192615018c7,67da0763_fcd4_4e68_85dd_3e3deac4f2ac
4,430ca3a7_9f16_490f_aae0_f7fd9675c8a9,05b18e56_48a1_4849_b3c0_693beb43bd53


Data saved to 'air_temperature_sensors.csv'


In [17]:
filename_ttl='datasets/bts_site_b_train/Site_B.ttl'

g = Graph().parse(filename_ttl)

# Define the SPARQL query to retrieve electrical power sensors ordered by meters
query_energy = """
SELECT ?meter ?sensor ?stream_id ?phase_count ?phases ?unit ?power_complexity ?power_flow
WHERE {
    ?sensor rdf:type brick:Electrical_Power_Sensor .
    ?meter rdf:type brick:Electrical_Meter .
    ?sensor brick:isPointOf ?meter .
    ?sensor senaps:stream_id ?stream_id .
    OPTIONAL { ?sensor brick:electricalPhaseCount [ brick:value ?phase_count ] . }
    OPTIONAL { ?sensor brick:electricalPhases [ brick:value ?phases ] . }
    OPTIONAL { ?sensor brick:hasUnit [ brick:value ?unit ] . }
    OPTIONAL { ?sensor brick:powerComplexity [ brick:value ?power_complexity ] . }
    OPTIONAL { ?sensor brick:powerFlow [ brick:value ?power_flow ] . }
}
ORDER BY ?meter
"""

# Run the query
results_energy = g.query(query_energy)

# Prepare the data for the DataFrame
data_energy = []
for row in results_energy:
    data_energy.append({
        'Meter': str(row['meter']),
        'Sensor': str(row['sensor']),
        'Stream ID': str(row['stream_id']),
        'Phase Count': str(row['phase_count']) if row['phase_count'] else None,
        'Phases': str(row['phases']) if row['phases'] else None,
        'Unit': str(row['unit']) if row['unit'] else None,
        'Power Complexity': str(row['power_complexity']) if row['power_complexity'] else None,
        'Power Flow': str(row['power_flow']) if row['power_flow'] else None
    })

# Create a DataFrame from the results
df_energy_meters = pd.DataFrame(data_energy)

# Display the DataFrame
# df_energy_meters.head()
display(df_energy_meters.head())

,Meter,Sensor,Stream ID,Phase Count,Phases,Unit,Power Complexity,Power Flow
0,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,d40fdd3a_ee77_485c_9c4f_7f074900c1fe,3,ABC,KiloV-A,apparent,net
1,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,969923d8_06fc_4a23_9559_0504ccee997a,3,ABC,KiloV-A_Reactive,reactive,net
2,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,87677553_3917_452b_8647_6d912927f558,3,ABC,KiloW,real,net
3,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,88f33bdd_5e78_48b1_b8e4_7ee6b358ea35,1,A,KiloW,real,net
4,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,7da0d8bf_05c0_4213_bd5d_3356917ad67d,1,B,KiloW,real,net


In [18]:
# Define the SPARQL query to retrieve Chilled_Water_Differential_Temperature sensors ordered by meters
query_chiller = """
SELECT ?meter ?sensor ?stream_id
WHERE {
    ?sensor rdf:type brick:Chilled_Water_Differential_Temperature_Sensor .
    ?meter rdf:type brick:Chiller .
    ?sensor brick:isPointOf ?meter .
    ?sensor senaps:stream_id ?stream_id
}
ORDER BY ?meter
"""

# Run the query
results_chiller = g.query(query_chiller)

# Prepare the data for the DataFrame
data_chiller = []
for row in results_chiller:
    data_chiller.append({
        'Meter': str(row['meter']),
        'Sensor': str(row['sensor']),
        'Stream ID': str(row['stream_id'])
    })

# Create a DataFrame from the results
df_chiller_meters = pd.DataFrame(data_chiller)

# Display the DataFrame
display(df_chiller_meters)

,Meter,Sensor,Stream ID
0,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,1ccf1c24_fca5_4b0d_af96_34522b52e810


In [ ]:
# For gas measurement

# Define the SPARQL query to retrieve Gas Meter sensors
query_gas = """
SELECT ?meter ?sensor ?stream_id
WHERE {
    ?sensor rdf:type brick:Usage_Sensor .
    ?meter rdf:type brick:Building_Gas_Meter .
    ?sensor brick:isPointOf ?meter .
    ?sensor senaps:stream_id ?stream_id
}
ORDER BY ?meter
"""

# Run the query
results_gas = g.query(query_gas)

# Prepare the data for the DataFrame
data_gas = []
for row in results_gas:
    data_gas.append({
        'Meter': str(row['meter']),
        'Sensor': str(row['sensor']),
        'Stream ID': str(row['stream_id'])
    })

# Create a DataFrame from the results
df_gas_meters = pd.DataFrame(data_gas)

In [ ]:
# For water measurement

# Define the SPARQL query to retrieve Water Meter sensors
query_water = """
SELECT ?meter ?sensor ?stream_id
WHERE {
    ?sensor rdf:type brick:Usage_Sensor .
    ?meter rdf:type brick:Building_Water_Meter .
    ?sensor brick:isPointOf ?meter .
    ?sensor senaps:stream_id ?stream_id
}
ORDER BY ?meter
"""

# Run the query
results_water = g.query(query_water)

# Prepare the data for the DataFrame
data_water = []
for row in results_water:
    data_water.append({
        'Meter': str(row['meter']),
        'Sensor': str(row['sensor']),
        'Stream ID': str(row['stream_id'])
    })

# Create a DataFrame from the results
df_water_meters = pd.DataFrame(data_water)

In [19]:
# Define the SPARQL query to retrieve Water-temperature sensors ordered by Hot water system meter
query_boiler = """
SELECT ?meter ?sensor ?stream_id
WHERE {
    ?sensor rdf:type brick:Water_Temperature_Sensor .
    ?meter rdf:type brick:Hot_Water_System .
    ?sensor brick:isPointOf ?meter .
    ?sensor senaps:stream_id ?stream_id
}
ORDER BY ?meter
"""

# Run the query
results_boiler = g.query(query_boiler)

# Prepare the data for the DataFrame
data_boiler = []
for row in results_boiler:
    data_boiler.append({
        'Meter': str(row['meter']),
        'Sensor': str(row['sensor']),
        'Stream ID': str(row['stream_id'])
    })

# Create a DataFrame from the results
df_boiler_meters = pd.DataFrame(data_boiler)

# Display the DataFrame
display(df_boiler_meters)

,Meter,Sensor,Stream ID
0,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,e022d248_71dc_4142_a4af_01f2eff3360b
